# Binary Text-Sentiment-Analysis using RoBERTa

In [1]:
!pip install transformers[torch] datasets comet_ml tensorboard evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.1/698.1 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.8/981.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.9 MB/

## 1. Download and Load the Dataset

The following command will download combined dataset from `IMDb Movie Dataset`.

In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!unzip imdb-dataset-of-50k-movie-reviews.zip

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 51% 13.0M/25.7M [00:00<00:00, 132MB/s]
100% 25.7M/25.7M [00:00<00:00, 160MB/s]
Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [26]:
import pandas as pd
import numpy as np

# Read dataset
df = pd.read_csv("IMDB Dataset.csv")

# Reset the index
df.reset_index(drop=True, inplace=True)
df.head(), df.shape

(                                              review sentiment
 0  One of the other reviewers has mentioned that ...  positive
 1  A wonderful little production. <br /><br />The...  positive
 2  I thought this was a wonderful way to spend ti...  positive
 3  Basically there's a family where a little boy ...  negative
 4  Petter Mattei's "Love in the Time of Money" is...  positive,
 (50000, 2))

## 2. Text Pre-Processing

- Cleaning up the text data by removing punctuation, extra spaces, and numbers.
- Transform sentences into individual words, remove common words (known as "stop words")

In [27]:
import re

# Precompile regular expressions for faster preprocessing
non_word_chars_pattern = re.compile(r"[^\w\s]")
whitespace_pattern = re.compile(r"\s+")
url_pattern = re.compile(r'http\S+|www\S+')
username_pattern = re.compile(r"@([^\s]+)")
hashtags_pattern = re.compile(r"#\d+")
br_pattern = re.compile(r'\s*')
contractions_pattern = re.compile(r"\b(can't|won't|n't|'re|'s|'d|'ll|'t|'ve|'m)\b")

# Expand common contractions
contractions_dict = {
    "can't": "cannot", "won't": "will not", "n't": "not", "'re": "are", "'s": "is",
    "'d": "would", "'ll": "will", "'t": "not", "'ve": "have", "'m": "am"
}

def expand_contractions(text):
    return contractions_pattern.sub(lambda x: contractions_dict.get(x.group()), text)

def preprocess_string(s):
    """Clean and preprocess the input string."""
    # Lowercase text
    s = s.lower()
    # Expand contractions
    s = expand_contractions(s)
    # Remove URLs
    s = url_pattern.sub('', s)
    # Remove usernames and hashtags
    s = username_pattern.sub('', s)
    s = hashtags_pattern.sub('', s)
    # Remove non-word characters (preserving letters and numbers only)
    s = non_word_chars_pattern.sub(' ', s)
    # Normalize whitespace
    s = whitespace_pattern.sub(' ', s)

    return s.strip()

In [28]:
from tqdm.notebook import tqdm_notebook

preprocessed_reviews = []

# Apply preprocessing
for review in tqdm_notebook(df['review'], desc='Preprocessing'):
    preprocessed_review = preprocess_string(review)
    preprocessed_reviews.append(preprocessed_review)

# Assign the preprocessed reviews back to 'review' column
df['review'] = preprocessed_reviews

Preprocessing:   0%|          | 0/50000 [00:00<?, ?it/s]

## 3. Mapping `sentiment` column to numeric values

In [29]:
# Map 'positive' to 1 & 'negative' to 0
df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})
df.head()

<ipython-input-29-5501ca7fd881>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'positive': 1, 'negative': 0})


,review,sentiment
0,one of the other reviewers has mentioned that ...,1
1,a wonderful little production br br the filmin...,1
2,i thought this was a wonderful way to spend ti...,1
3,basically thereis a family where a little boy ...,0
4,petter matteiis love in the time of money is a...,1


## 4. Spliiting datasets into train and test

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['review'],
                                                    df['sentiment'],
                                                    test_size=0.2)

len(X_train), len(X_test)

(40000, 10000)

In [31]:
X_train, X_test, y_train, y_test = list(X_train), list(X_test), list(y_train), list(y_test)
X_train[:2], y_train[:2]

(['im hoping this was made before half past dead and exit wounds because it was rubbish seagal wasnt to blame it was down to the crap directing when the few action scenes took place the plot was also confusing and basically just felt rushed out maybe it was shelved and released to capitalise on seagals newer films br br 3 10 br br heis not through yet bring on under siege 3 and loose some weight',
  'bwana devil is reputedly the first major studio full length feature filmed entirely in the 3d process supposedly producer oboler went to africa to shoot a different movie but after hearing the tale of two man eating lions terrorizing railway builders decided on this one itis a good story too almost hemmingway like fear redemption the great white hunter and all itis the telling of the story that seems to drag almost as though filming in the new process was too weighty for the crew the action scenes are stiff almost too staged but these technical problems appear small in light of the filmis 

## 5. Preparing data using custom dataloader

In [37]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Setting device agnostic code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [10]:
class data(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, index):
    item = {key: torch.tensor(val[index]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[index])
    return item

  def __len__(self):
    return len(self.labels)

## 6. Load PreTrained RoBERTa Model

In [11]:
from huggingface_hub import notebook_login

# Paste hugging face token with write permission enabled and log in
notebook_login()

In [33]:
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name, model_max_length=384)

## 7. Tokenize and Create Encoded Dataset

In [34]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

# Tokenize with truncation and padding and create dataset from tokenized data
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)

train_dataset = data(train_encoding, y_train)
test_dataset = data(test_encoding, y_test)

## 8. Fine-Tuning RoBERTa-base

In [35]:
import comet_ml
from comet_ml import Experiment

comet_ml.login(project_name="sentiment-analysis-transformer")

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [38]:
batch_size = 32
epochs = 5

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-sentiment",
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=1,                   # adjust if needed for larger batch sizes
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    num_train_epochs=epochs,                         # specify the number of epochs
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="epoch",                           # Perform evaluation at the end of each epoch
    per_device_eval_batch_size=batch_size,
    save_strategy="epoch",                           # Save model at the end of each epoch
    save_total_limit=1,                              # Only keep the best model (limit to 1 checkpoint)
    logging_strategy="epoch",
    report_to=["comet_ml", "tensorboard"],           # Experiment Tracker: CometML or others
    load_best_model_at_end=True,                     # Load the best model at the end of training
    metric_for_best_model="accuracy",               # Use eval_loss as the metric to track the best model
    greater_is_better=True,                         # Lower eval_loss is better
    push_to_hub=True,                                # Automatically push the best model to Hugging Face Hub
)

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=2,
    early_stopping_threshold=0.0
)

## 9. Train the Fine-Tuned BERT Model

In [39]:
import evaluate
from sklearn.metrics import confusion_matrix

accuracy_metric = evaluate.load("accuracy")

LABELS = ['negative', 'positive']
exp = comet_ml.Experiment()

# Compute_metrics function with confusion matrix logging
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)

    # Calculate confusion matrix
    cm = confusion_matrix(labels, predictions)

    # Log the confusion matrix to Comet ML
    exp.log_confusion_matrix(matrix=cm, labels=LABELS, file_name="confusion-matrix.json")

    return accuracy

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch, keras, sklearn, tensorflow.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/luluw8071/sentiment-analysis-transformer/4eb12a41c078438bb83d21667e833f67

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [41]:
# Label mapping
label_mapping = {0: 'negative', 1: 'positive'}

model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Override the model configuration for custom labels
model.config.id2label = label_mapping
model.config.label2id = {v: k for k, v in label_mapping.items()}

trainer = Trainer(
    model=model,                        # The instantiated Transformers model to be trained
    args=training_args,                 # Training arguments, defined above
    train_dataset=train_dataset,        # Training dataset
    eval_dataset=test_dataset,          # Evaluation dataset
    tokenizer=tokenizer,                # Tokenizer
    data_collator=data_collator,        # Data collator
    compute_metrics=compute_metrics,    # Function to compute metrics
    callbacks=[early_stopping_callback] # Early Stop Callback
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-41-6a0604644c81>:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
from accelerate import Accelerator

# Initialize Accelerator and Trainer
Accelerator()
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.281500,0.170523,0.936600
2,0.135800,0.154979,0.946300
3,0.087900,0.208142,0.947000
4,0.056400,0.247934,0.947400
5,0.033900,0.259457,0.949500


TrainOutput(global_step=6250, training_loss=0.11910875, metrics={'train_runtime': 4675.5952, 'train_samples_per_second': 42.775, 'train_steps_per_second': 1.337, 'total_flos': 3.94670126592e+16, 'train_loss': 0.11910875, 'epoch': 5.0})

In [ ]:
exp.end()

In [45]:
trainer.evaluate()

{'eval_loss': 0.25945743918418884,
 'eval_accuracy': 0.9495,
 'eval_runtime': 51.399,
 'eval_samples_per_second': 194.556,
 'eval_steps_per_second': 6.09,
 'epoch': 5.0}

In [47]:
kwargs = {
    "dataset": "imdb-dataset-of-50k-movie-reviews",
    # "dataset_args": "config: hi, split: test",
    "language": "en",
    "finetuned_from": model_name,
    "tasks": "sentiment-classification",
}

trainer.push_to_hub(**kwargs)

events.out.tfevents.1731906143.97a70b663b7c.500.1:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/luluw/roberta-base-finetuned-sentiment/commit/3719ee502f63a28f58b581b6007f902bb3933076', commit_message='End of training', commit_description='', oid='3719ee502f63a28f58b581b6007f902bb3933076', pr_url=None, repo_url=RepoUrl('https://huggingface.co/luluw/roberta-base-finetuned-sentiment', endpoint='https://huggingface.co', repo_type='model', repo_id='luluw/roberta-base-finetuned-sentiment'), pr_revision=None, pr_num=None)

## 10. Sentiment Prediction using custom text


In [48]:
# Tokenize text, get output from model and predict
def predict_sentiment(model, tokenizer, text, device):
    tokenized = tokenizer(text, truncation=True, padding=True, return_tensors='pt').to(device)
    outputs = model(**tokenized)

    probs = F.softmax(outputs.logits, dim=-1)
    preds = torch.argmax(outputs.logits, dim=-1).item()
    probs_max = probs.max().detach().cpu().numpy()

    prediction = "Positive" if preds == 1 else "Negative"
    print(f'{text}\nSentiment: {prediction}\tProbability: {probs_max*100:.2f}%\n', end="-"*50 + "\n")
    # return prediction, probs_max

In [50]:
# An example of complex review that contains both positive and negative sentiment
texts = ["Despite facing numerous challenges and setbacks, the team worked tirelessly and managed to exceed all expectations, achieving remarkable success. However, despite their best efforts, the project encountered multiple setbacks, ultimately leading to its failure and significant financial losses.",
         "The hotel room was clean and comfortable, and the amenities were well-maintained. However, the noise from the nearby construction site was disruptive due to which i could not focus when working.",
         "The movie had an intriguing plot and captivating visuals, but the sound quality was poor, making it difficult to fully enjoy the experience."]

for text in texts:
  predict_sentiment(model, tokenizer, text, device)

Despite facing numerous challenges and setbacks, the team worked tirelessly and managed to exceed all expectations, achieving remarkable success. However, despite their best efforts, the project encountered multiple setbacks, ultimately leading to its failure and significant financial losses.
Sentiment: Negative	Probability: 62.80%
--------------------------------------------------
The hotel room was clean and comfortable, and the amenities were well-maintained. However, the noise from the nearby construction site was disruptive due to which i could not focus when working.
Sentiment: Negative	Probability: 99.00%
--------------------------------------------------
The movie had an intriguing plot and captivating visuals, but the sound quality was poor, making it difficult to fully enjoy the experience.
Sentiment: Negative	Probability: 68.71%
--------------------------------------------------


In [51]:
# Breaking down above example into parts
texts = ["Despite facing numerous challenges and setbacks, the team worked tirelessly and managed to exceed all expectations, achieving remarkable success.",
         "However, despite their best efforts, the project encountered multiple setbacks, ultimately leading to its failure and significant financial losses.",
         "The hotel room was clean and comfortable, and the amenities were well-maintained. However, the noise from the nearby construction site was disruptive due to which i could not focus when working."]

for text in texts:
  predict_sentiment(model, tokenizer, text, device)

Despite facing numerous challenges and setbacks, the team worked tirelessly and managed to exceed all expectations, achieving remarkable success.
Sentiment: Positive	Probability: 99.64%
--------------------------------------------------
However, despite their best efforts, the project encountered multiple setbacks, ultimately leading to its failure and significant financial losses.
Sentiment: Negative	Probability: 70.44%
--------------------------------------------------
The hotel room was clean and comfortable, and the amenities were well-maintained. However, the noise from the nearby construction site was disruptive due to which i could not focus when working.
Sentiment: Negative	Probability: 99.00%
--------------------------------------------------


## Load the fine-tuned model from hugging face

In [ ]:
%%writefile inference.py
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

model_name = "luluw/roberta-base-finetuned-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(
    model_name,
    max_length=384,
    num_labels=2
)

# Example usage
text = "Absolutely hated the product. Waste of money"
inputs = tokenizer(text, return_tensors='pt')
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=-1)

print(text)
print("positive" if predictions == 1 else "negative")

Overwriting inference.py


In [ ]:
!python3 inference.py